**Collaborative Filtering Approaches**
1. Memory-based:    ItemKNN
2. Model-based:     BPR, LightGCN
3. Context-based:   FM, DeepFM, WideDeep

**Content-based Approaches:** TFIDF (Cornac Models)

**Knowledge-based Approaches:** KGCN, KGAT, KGIN

**Hybrid Systems:** NeuMF

In [14]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from recbole.model.knowledge_aware_recommender import KGCN, KGIN, KGAT
from recbole.model.general_recommender import BPR, LightGCN, ItemKNN, NeuMF


from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer, get_model

# configurations initialization
config = Config(model='BPR', dataset='ml-100k')

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

# write config info into log
logger.info(config)

# dataset creating and filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

# model loading and initialization
model = BPR(config, train_data._dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer_class = get_trainer(config["MODEL_TYPE"], config["model"])
trainer = trainer_class(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

# model evaluation
test_result = trainer.evaluate(test_data)
print(test_result)

21 Jul 01:39    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = C:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\Lib\site-packages\recbole\config\../dataset_example/ml-100k
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_met

OrderedDict([('recall@10', 0.2388), ('mrr@10', 0.482), ('ndcg@10', 0.2862), ('hit@10', 0.772), ('precision@10', 0.1914)])


In [7]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation


from recbole.model.general_recommender import BPR, LightGCN, ItemKNN, NeuMF
from recbole.model.context_aware_recommender import FM, DeepFM, WideDeep
from recbole.model.knowledge_aware_recommender import KGCN, KGIN, KGAT

from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer

models = {
    'BPR': BPR,
    # 'LightGCN': LightGCN,
    # 'ItemKNN': ItemKNN, 
    # 'FM': FM,
    # 'DeepFM': DeepFM,
    # 'WideDeep': WideDeep,
    # 'KGCN': KGCN,
    # 'KGIN': KGIN,
    # 'KGAT': KGAT,
    # 'NeuMF': NeuMF
    }

for model_name, model_class in models.items():
    print(f"\033[93mRunning model: {model_name}\033[0m")

    

    config = Config(model=model_name, dataset='ml-100k')

    hp = HyperTuning(objective_function=objective_function, algo='exhaustive', early_stop=10,
                max_evals=100, params_file='params.hyper', fixed_config_file_list=config)

    # run
    hp.run()
    # export result to the file
    hp.export_result(output_file='hyper_example.result')
    # print best parameters
    print('best params: ', hp.best_params)
    # print best result
    print('best result: ')
    print(hp.params2result[hp.params2str(hp.best_params)])


    # init random seed
    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    model = model_class(config, train_data._dataset).to(config['device'])
    logger.info(model)

    # trainer loading and initialization
    trainer_class = get_trainer(config["MODEL_TYPE"], config["model"])
    trainer = trainer_class(config, model)

    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

    # model evaluation
    test_result = trainer.evaluate(test_data)
    print(test_result)

Running model: BPR
ERROR in rec_eval                                    
EXCEPTION                                            
<class 'AttributeError'>                             
'numpy.random.mtrand.RandomState' object has no attribute 'integers'
NODE                                                 
0 randint                                            
1   Literal{2}
2  size =
3   len
4     array_union
5       array_union
6         array_union
7           Literal{new_ids}
8  rng =
9   Literal{rng-placeholder}
  0%|          | 0/6 [00:00<?, ?trial/s, best loss=?]


AttributeError: 'numpy.random.mtrand.RandomState' object has no attribute 'integers'

In [ ]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from recbole.model.general_recommender import BPR, LightGCN, ItemKNN, NeuMF
from recbole.model.context_aware_recommender import FM, DeepFM, WideDeep
from recbole.model.knowledge_aware_recommender import KGCN, KGIN, KGAT

from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer

models = {
    'BPR': BPR,
    # other models commented out
}

for model_name, model_class in models.items():
    print(f"\033[93mRunning model: {model_name}\033[0m")

    config = Config(model=model_name, dataset='ml-100k')
    config_dict = {'model': model_name, 'dataset': 'ml-100k'}
    # Initialize hyperparameter tuning for the model
    hp = HyperTuning(
        objective_function=objective_function,
        algo='exhaustive',
        early_stop=10,
        max_evals=100,
        params_file='params.hyper',
        fixed_config_file_list=['config.yaml']  # Pass config object here
        # fixed_config_file_list=[config]  # Pass config object here
    )

    # Run the hyperparameter tuning
    hp.run()
    hp.export_result(output_file='hyper_example.result')

    print('best params: ', hp.best_params)
    print('best result: ')
    print(hp.params2result[hp.params2str(hp.best_params)])

    # Now run training with best hyperparameters
    

    # Seed and logger
    init_seed(config['seed'], config['reproducibility'])
    init_logger(config)
    logger = getLogger()
    logger.info(config)

    # Dataset loading and preparation
    dataset = create_dataset(config)
    logger.info(dataset)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # Model initialization
    model = model_class(config, train_data._dataset).to(config['device'])
    logger.info(model)

    # Trainer initialization and training
    trainer_class = get_trainer(config["MODEL_TYPE"], config["model"])
    trainer = trainer_class(config, model)
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

    # Evaluation
    test_result = trainer.evaluate(test_data)
    print(test_result)


Running model: BPR
running parameters:                                  
{'embedding_size': 64, 'learning_rate': 0.0009845219514902307, 'mlp_hidden_size': '[128,128]'}
  0%|          | 0/6 [00:00<?, ?trial/s, best loss=?]

c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)

c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate ob

current best valid score: 0.3896                     
current best valid result:                           
OrderedDict([('recall@10', 0.2078), ('mrr@10', 0.3896), ('ndcg@10', 0.2294), ('hit@10', 0.7391), ('precision@10', 0.1572)])
current test result:                                 
OrderedDict([('recall@10', 0.2447), ('mrr@10', 0.4889), ('ndcg@10', 0.2917), ('hit@10', 0.7805), ('precision@10', 0.1949)])
running parameters:                                                 
{'embedding_size': 128, 'learning_rate': 0.08179343529093307, 'mlp_hidden_size': '[64,64,64]'}
 17%|█▋        | 1/6 [00:31<02:38, 31.78s/trial, best loss: -0.3896]

c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)

c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate ob

running parameters:                                                 
{'embedding_size': 64, 'learning_rate': 0.8723162187044113, 'mlp_hidden_size': '[64,64,64]'}
 33%|███▎      | 2/6 [00:55<01:47, 26.75s/trial, best loss: -0.3896]

...

100%|██████████| 6/6 [01:43<00:00, 17.24s/trial, best loss: -0.3896]

21 Jul 02:29    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = C:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\Lib\site-packages\recbole\config\../dataset_example/ml-100k
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_met


best params:  {'embedding_size': 64, 'learning_rate': 0.0009845219514902307, 'mlp_hidden_size': '[128,128]'}
best result: 
{'model': 'BPR', 'best_valid_score': 0.3896, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.2078), ('mrr@10', 0.3896), ('ndcg@10', 0.2294), ('hit@10', 0.7391), ('precision@10', 0.1572)]), 'test_result': OrderedDict([('recall@10', 0.2447), ('mrr@10', 0.4889), ('ndcg@10', 0.2917), ('hit@10', 0.7805), ('precision@10', 0.1949)])}


c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate obj

OrderedDict([('recall@10', 0.2388), ('mrr@10', 0.482), ('ndcg@10', 0.2862), ('hit@10', 0.772), ('precision@10', 0.1914)])


In [2]:
# @Time   : 2023/2/13
# @Author : Gaowei Zhang
# @Email  : zgw2022101006@ruc.edu.cn


import argparse

from recbole.quick_start import run
from recbole.utils import list_to_latex

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--model_list", "-m", type=str, default="BPR", help="name of models"
    )
    parser.add_argument(
        "--dataset", "-d", type=str, default="ml-100k", help="name of datasets"
    )
    parser.add_argument("--config_files", type=str, default=None, help="config files")
    parser.add_argument(
        "--valid_latex", type=str, default="./latex/valid.tex", help="config files"
    )
    parser.add_argument(
        "--test_latex", type=str, default="./latex/test.tex", help="config files"
    )
    parser.add_argument(
        "--nproc", type=int, default=1, help="the number of process in this group"
    )
    parser.add_argument(
        "--ip", type=str, default="localhost", help="the ip of master node"
    )
    parser.add_argument(
        "--port", type=str, default="5678", help="the port of master node"
    )
    parser.add_argument(
        "--world_size", type=int, default=-1, help="total number of jobs"
    )
    parser.add_argument(
        "--group_offset",
        type=int,
        default=0,
        help="the global rank offset of this group",
    )

    args, _ = parser.parse_known_args()

    model_list = args.model_list.strip().split(",")
    config_file_list = (
        args.config_files.strip().split(" ") if args.config_files else None
    )
    valid_file = args.valid_latex.strip()
    test_file = args.test_latex.strip()

    valid_result_list = []
    test_result_list = []

    run_times = len(model_list)

    for idx in range(run_times):
        model = model_list[idx]

        valid_res_dict = {"Model": model}
        test_res_dict = {"Model": model}
        result = run(
            model,
            args.dataset,
            config_file_list=config_file_list,
            nproc=args.nproc,
            world_size=args.world_size,
            ip=args.ip,
            port=args.port,
            group_offset=args.group_offset,
        )
        valid_res_dict.update(result["best_valid_result"])
        test_res_dict.update(result["test_result"])
        bigger_flag = result["valid_score_bigger"]
        subset_columns = list(result["best_valid_result"].keys())

        valid_result_list.append(valid_res_dict)
        test_result_list.append(test_res_dict)

    df_valid, tex_valid = list_to_latex(
        convert_list=valid_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )
    df_test, tex_test = list_to_latex(
        convert_list=test_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )

    with open(valid_file, "w") as f:
        f.write(tex_valid)
    with open(test_file, "w") as f:
        f.write(tex_test)

In [ ]:
def main():
    # Set values directly
    model_list = "BPR".strip().split(",")
    dataset = "ml-100k"
    config_file_list = None  # or ["path/to/config1.yaml", "path/to/config2.yaml"]
    valid_file = "./latex/valid.tex"
    test_file = "./latex/test.tex"
    nproc = 1
    ip = "localhost"
    port = "5678"
    world_size = -1
    group_offset = 0

    valid_result_list = []
    test_result_list = []

    run_times = len(model_list)

    for idx in range(run_times):
        model = model_list[idx]

        valid_res_dict = {"Model": model}
        test_res_dict = {"Model": model}

        result = run(
            model,
            dataset,
            config_file_list=config_file_list,
            nproc=nproc,
            world_size=world_size,
            ip=ip,
            port=port,
            group_offset=group_offset,
        )

        valid_res_dict.update(result["best_valid_result"])
        test_res_dict.update(result["test_result"])
        bigger_flag = result["valid_score_bigger"]
        subset_columns = list(result["best_valid_result"].keys())

        valid_result_list.append(valid_res_dict)
        test_result_list.append(test_res_dict)

    df_valid, tex_valid = list_to_latex(
        convert_list=valid_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )
    df_test, tex_test = list_to_latex(
        convert_list=test_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )

    # with open(valid_file, "w") as f:
    #     f.write(tex_valid)
    # with open(test_file, "w") as f:
    #     f.write(tex_test)


In [11]:
main()

Running model: BPR
running parameters:                                  
{'embedding_size': 128, 'learning_rate': 0.018319142664538036, 'mlp_hidden_size': '[64,64,64]'}
  0%|          | 0/6 [00:00<?, ?trial/s, best loss=?]

c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)

c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate ob

current best valid score: 0.3111                     
current best valid result:                           
OrderedDict([('recall@10', 0.1525), ('mrr@10', 0.3111), ('ndcg@10', 0.1673), ('hit@10', 0.6373), ('precision@10', 0.1142)])
current test result:                                 
OrderedDict([('recall@10', 0.1701), ('mrr@10', 0.3532), ('ndcg@10', 0.1985), ('hit@10', 0.666), ('precision@10', 0.1375)])
running parameters:                                                 
{'embedding_size': 128, 'learning_rate': 0.14017820619383206, 'mlp_hidden_size': '[128,128]'}
 17%|█▋        | 1/6 [00:07<00:36,  7.35s/trial, best loss: -0.3111]

c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)

c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate ob

running parameters:                                                 
{'embedding_size': 64, 'learning_rate': 0.0033582847266069734, 'mlp_hidden_size': '[128,128]'}
 33%|███▎      | 2/6 [00:19<00:41, 10.42s/trial, best loss: -0.3111]

...

21 Jul 12:35    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = C:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\Lib\site-packages\recbole\config\../dataset_example/ml-100k
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.000851776899960467
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = M


Best Params:  {'embedding_size': 96, 'learning_rate': 0.000851776899960467, 'mlp_hidden_size': '[64,64,64]'}
Best Result: 
{'model': 'BPR', 'best_valid_score': 0.3866, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.2171), ('mrr@10', 0.3866), ('ndcg@10', 0.2329), ('hit@10', 0.7582), ('precision@10', 0.1612)]), 'test_result': OrderedDict([('recall@10', 0.252), ('mrr@10', 0.4973), ('ndcg@10', 0.2973), ('hit@10', 0.7869), ('precision@10', 0.2005)])}


c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
c:\Users\Sharjeel Mustafa\miniconda3\envs\recbole\lib\site-packages\recbole\data\dataset\dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate obj


Test Result:
OrderedDict([('recall@10', 0.2536), ('mrr@10', 0.4873), ('ndcg@10', 0.2934), ('hit@10', 0.807), ('precision@10', 0.2002)])


NameError: name 'list_to_latex' is not defined

In [10]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from recbole.model.general_recommender import BPR, LightGCN, ItemKNN, NeuMF
from recbole.model.context_aware_recommender import FM, DeepFM, WideDeep
from recbole.model.knowledge_aware_recommender import KGCN, KGIN, KGAT

from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer

def main():
    # === Configuration ===
    model_list = ["BPR"]
    dataset_name = "ml-100k"
    fixed_config_files = [r"C:\Users\Sharjeel Mustafa\Documents\Documents\Academic\C874\C874-final-project\config\config.yaml"]
    valid_file = "./latex/valid.tex"
    test_file = "./latex/test.tex"

    valid_result_list = []
    test_result_list = []

    for model_name in model_list:
        print(f"\033[93mRunning model: {model_name}\033[0m")

        # Step 1: Hyperparameter Tuning
        hp = HyperTuning(
            objective_function=objective_function,
            algo='exhaustive',
            early_stop=10,
            max_evals=100,
            params_file='C:/Users/Sharjeel Mustafa/Documents/Documents/Academic/C874/C874-final-project/config/model.hyper',
            fixed_config_file_list=fixed_config_files
        )

        hp.run()
        hp.export_result(output_file='hyper_example.result')

        print("Best Params: ", hp.best_params)
        print("Best Result: ")
        print(hp.params2result[hp.params2str(hp.best_params)])

        # Step 2: Re-train using best parameters
        config = Config(model=model_name, dataset=dataset_name, config_dict=hp.best_params)
        init_seed(config["seed"], config["reproducibility"])
        init_logger(config)
        logger = getLogger()
        logger.info(config)

        dataset = create_dataset(config)
        logger.info(dataset)

        train_data, valid_data, test_data = data_preparation(config, dataset)

        model_class = config['model']
        model = config.model_class(config, train_data._dataset).to(config['device'])
        logger.info(model)

        trainer_class = get_trainer(config["MODEL_TYPE"], config["model"])
        trainer = trainer_class(config, model)

        best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
        test_result = trainer.evaluate(test_data)

        # For LaTeX summary
        valid_result_list.append({"Model": model_name, **best_valid_result})
        test_result_list.append({"Model": model_name, **test_result})
        bigger_flag = config["valid_metric_bigger"]
        subset_columns = list(best_valid_result.keys())

        print("\nTest Result:")
        print(test_result)

    # Step 3: Generate and write LaTeX outputs
    df_valid, tex_valid = list_to_latex(
        convert_list=valid_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )
    df_test, tex_test = list_to_latex(
        convert_list=test_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )

    with open(valid_file, "w") as f:
        f.write(tex_valid)
    with open(test_file, "w") as f:
        f.write(tex_test)
